# Markowitz Efficient Frontier

## 1. Imports

In [135]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from tqdm import tqdm
from fetchData import fetch_raw_data_yf, get_matrices, getNasdaqStocks

## 2. Fetch Data

### Get all Nasdaq Stocks

In [136]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT",    # Lockheed Martin Corporation
    "GS",   # Goldman Sachs Group, Inc.
    "MS",   # Morgan Stanley
    "IBM",  # International Business Machines Corporation
    "GE",   # General Electric Company
    "F",    # Ford Motor Company
    "GM",   # General Motors Company
    "UBER", # Uber Technologies, Inc.
    "LYFT", # Lyft, Inc.
    "SNAP", # Snap Inc.
    "TWTR", # Twitter, Inc.
    "SPOT", # Spotify Technology S.A.
    "AMD",  # Advanced Micro Devices, Inc.
    "TXN",  # Texas Instruments Incorporated
    "BABA", # Alibaba Group Holding Limited
    "SAP",  # SAP SE
    "HON",  # Honeywell International Inc.
    "BA",   # Boeing Company
    "RTX",  # Raytheon Technologies Corporation
    "CAT",  # Caterpillar Inc.
    "DE",   # Deere & Company
    "MMM",  # 3M Company
    "DUK",  # Duke Energy Corporation
    "SO",   # Southern Company
    "EXC",  # Exelon Corporation
    "NEE",  # NextEra Energy, Inc.
    "AEP",  # American Electric Power Company, Inc.
    "SRE",  # Sempra Energy
    "ETN",  # Eaton Corporation plc
    "EMR",  # Emerson Electric Co.
    "SYY",  # Sysco Corporation
    "KR",   # Kroger Co.
    "GIS",  # General Mills, Inc.
    "K",    # Kellogg Company
    "CPB",  # Campbell Soup Company
    "MO",   # Altria Group, Inc.
    "PM",   # Philip Morris International Inc.
    "BTI",  # British American Tobacco plc
    "RDY",  # Dr. Reddy's Laboratories Ltd.
    "GILD", # Gilead Sciences, Inc.
    "BIIB", # Biogen Inc.
    "CELG", # Celgene Corporation
    "AMGN", # Amgen Inc.
    "SYK",  # Stryker Corporation
    "BSX",  # Boston Scientific Corporation
    "ISRG", # Intuitive Surgical, Inc.
    "ZBH",  # Zimmer Biomet Holdings, Inc.
    "EW",   # Edwards Lifesciences Corporation
    "RMD",  # ResMed Inc.
    "VRTX", # Vertex Pharmaceuticals Incorporated
    "REGN",  # Regeneron Pharmaceuticals, Inc.
]

len(assets)

100

In [137]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets)

max_combination

[*********************100%***********************]  99 of 99 completed

5 Failed downloads:
['SPOT', 'LYFT', 'UBER']: YFPricesMissingError('possibly delisted; no price data found  (1d 2015-01-01 -> 2018-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1420088400, endDate = 1514782800")')
['CELG', 'TWTR']: YFTzMissingError('possibly delisted; no timezone found')


Omitted assets ( 5 ):  ['SPOT', 'CELG', 'LYFT', 'UBER', 'TWTR']
Time to fetch data: 3.83 seconds


94

## 3. Mean, Volatility and Covariance

In [138]:
annualized_returns = get_matrices(raw_data, max_combination ,None)[0][2]
cov = get_matrices(raw_data, max_combination ,None)[0][1]
names = get_matrices(raw_data, max_combination ,None)[0][0]
volatility = np.sqrt(np.diag(cov)) #standard deviation

risk_free_rate=0 
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

cov

Ticker,RDY_Close,SYY_Close,RMD_Close,GE_Close,XOM_Close,BAC_Close,SAP_Close,F_Close,ISRG_Close,ETN_Close,...,BMY_Close,DE_Close,QCOM_Close,GS_Close,GM_Close,ZBH_Close,ADBE_Close,CSCO_Close,T_Close,WFC_Close
Ticker,,,,,,,,,,,,,,,,,,,,,
RDY_Close,0.091301,0.004664,0.003514,0.009759,0.001098,0.014203,0.007040,0.007403,0.002827,0.006077,...,0.007134,0.004547,0.002038,0.011597,0.005315,0.008850,0.006747,0.008298,-0.000990,0.011404
SYY_Close,0.004664,0.026035,0.002990,0.002933,0.000840,0.006745,0.001352,0.007092,-0.002382,0.002456,...,0.003280,0.003719,-0.000142,0.005501,0.003176,0.002062,-0.003289,0.003550,0.006120,0.006026
RMD_Close,0.003514,0.002990,0.040309,-0.000520,0.000990,0.005972,0.008995,0.001188,0.011196,0.008634,...,0.004782,0.005981,0.004854,0.006470,0.003673,0.012785,0.008384,0.007801,0.003123,0.005680
GE_Close,0.009759,0.002933,-0.000520,0.045351,0.004385,0.012808,-0.000474,0.010366,-0.000750,0.008314,...,0.000852,0.007423,0.003324,0.009991,0.009873,0.002264,0.002537,0.004596,0.012486,0.008003
XOM_Close,0.001098,0.000840,0.000990,0.004385,0.010788,0.005378,0.000115,0.004791,-0.001245,0.004406,...,-0.000396,0.003418,0.000856,0.005698,0.003259,0.001363,-0.001663,0.001925,0.004238,0.004602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZBH_Close,0.008850,0.002062,0.012785,0.002264,0.001363,0.005950,0.002201,0.004220,0.005598,0.006093,...,0.001717,0.004610,-0.004816,0.005182,0.005340,0.037735,0.006898,0.005034,0.002133,0.004165
ADBE_Close,0.006747,-0.003289,0.008384,0.002537,-0.001663,-0.000410,0.013725,-0.001899,0.022981,0.006826,...,0.006387,0.004204,0.013411,0.002420,0.000990,0.006898,0.051898,0.008799,-0.004105,0.000741
CSCO_Close,0.008298,0.003550,0.007801,0.004596,0.001925,0.008951,0.006886,0.006799,0.005520,0.008125,...,0.007259,0.008686,0.009252,0.007960,0.009665,0.005034,0.008799,0.026896,0.004894,0.006977


In [139]:
hover_texts = [
    f"{ticker} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility, 
    y=annualized_returns, 
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Markowitz Mean Varience Model',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()


In [140]:
all_portfolios = []
dominant_portfolios = []

for _ in tqdm(range(10000)):
    next_i = False
    while True:
        temp = np.random.randint(3, 6)
        assets = np.random.choice(list(raw_data.columns), temp, replace=False)
        weights = np.random.rand(temp)
        weights /= weights.sum()

        curr_variance = 0
        curr_return = 0
        for i in range(len(assets)):
            curr_return += weights[i] * annualized_returns.loc[assets[i]]
            for j in range(len(assets)):
                curr_variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
        
        # Store everything in a dictionary
        all_portfolios.append({
            "return": curr_return,
            "variance": curr_variance,
            "tickers": assets,
            "weights": weights
        })

        # Check for dominated dominant_portfolios
        for portfolio in dominant_portfolios:
            if (portfolio["return"] > curr_return) and (portfolio["variance"] < curr_variance):
                next_i = True
                break

        if next_i:
            break

        # Store dominant in dictionary in a dictionary
        dominant_portfolios.append({
            "return": curr_return,
            "variance": curr_variance,
            "tickers": assets,
            "weights": weights
        })

        break

100%|██████████| 10000/10000 [00:02<00:00, 4846.23it/s]


In [141]:
len(dominant_portfolios)

104

In [142]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in all_portfolios],
    y=[p["return"] for p in all_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (p["variance"]**0.5) for p in all_portfolios],
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.2%}<br>Volatility: {p['variance']**0.5:.2f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.2f}" for i in range(len(p['tickers']))])
        for p in all_portfolios
    ]
))

fig1.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios'
)

fig1.show()

In [143]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=[p["variance"]**0.5 for p in dominant_portfolios],  # Convert variance to volatility
    y=[p["return"] for p in dominant_portfolios],
    mode='markers',
    marker=dict(
        color=[p["return"] / (p["variance"]**0.5) for p in dominant_portfolios],  # Sharpe Ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=[
        f"Return: {p['return']:.2%}<br>Volatility: {p['variance']**0.5:.2f}<br>" +
        "<br>".join([f"{p['tickers'][i]}: Weight={p['weights'][i]:.2f}" for i in range(len(p['tickers']))])
        for p in dominant_portfolios
    ]
))


fig2.add_trace(go.Scatter(
    x=volatility, 
    y=annualized_returns,
    mode='markers',
    hoverinfo='text',
    hovertext=[
        f"{name} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
        for name, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
    ],
    marker=dict(
        color='brown',
        size=5,
        symbol='triangle-up',  # Sets the marker shape to a triangle
        line=dict(width=1)
    ),
    name="Individual Assets"
))

fig2.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
)

fig2.show()